In [1]:
import pandas as pd
import numpy as np
from matplotlib.widgets import Slider
import matplotlib.pyplot as plt
import tkinter as tk
import os
import ipywidgets as widgets
from IPython.display import display
import seaborn as sns
#local module
import dataproject

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

pd.options.mode.chained_assignment = None
plt.style.use('seaborn-whitegrid')

In [2]:
#Open the data file and taking a look at it
Data = 'Data.xlsx'
pd.read_excel(Data).head(5)

,"Uddannelsesaktivitet efter status, bopælsområde, køn, uddannelse, alder og tid",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Enhed: antal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
2,Fuldført,Hele landet,Mænd og kvinder i alt,H10 Grundskole,Alder i alt,93865.0,95634.0,94320.0,100198.0,100715.0,102939.0,105233.0,106509.0,104984.0,103470.0,102982.0,103751.0,100831.0,100202.0,101043.0
3,NaN,NaN,NaN,NaN,-5 år,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,29.0
4,NaN,NaN,NaN,NaN,6 år,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,20.0,81.0


In [3]:
#Removing the first two rows as they are useless.
df = pd.read_excel(Data, skiprows=2)


#Removing first 2 columns
drop_these = ['Unnamed: 0', 'Unnamed: 1']
df.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made

In [4]:
#Renaming the columns
df.rename(columns = {'Unnamed: 2':'sex'}, inplace=True)
df.rename(columns = {'Unnamed: 3':'education'}, inplace=True)
df.rename(columns = {'Unnamed: 4':'age'}, inplace=True)

In [5]:
#find index of sex
total = df.loc[df["sex"]=="Mænd og kvinder i alt"].index
male = df.loc[df["sex"]=="Mænd"].index
female = df.loc[df["sex"]=="Kvinder"].index
#rename sex to total, male, and female
df["sex"][total] = "total"
df["sex"][male] = "male"
df["sex"][female] = "female"

In [6]:
#rename nan in sex to floats.
for i in range(0,len(df)):
    if type(df["sex"][i]) == str:
        x = df["sex"][i]
    else:
        df["sex"][i] = x

#rename nan in Education to floats
for i in range(0,len(df)):
    if type(df["education"][i]) == str:
        x = df["education"][i]
    else:
        df["education"][i] = x

In [7]:
#Formatting the dataframe to long so it's easier to work with in the interactive table.
df_long = pd.melt(df,id_vars=["sex","education","age"], var_name="year",value_name="uddannelsesaktivitet")
#Checking data
print(df_long)

sex                        education          age  year  \
0       total                   H10 Grundskole  Alder i alt  2005   
1       total                   H10 Grundskole        -5 år  2005   
2       total                   H10 Grundskole         6 år  2005   
3       total                   H10 Grundskole         7 år  2005   
4       total                   H10 Grundskole         8 år  2005   
...       ...                              ...          ...   ...   
18340  female  H80 Ph.d. og forskeruddannelser        38 år  2019   
18341  female  H80 Ph.d. og forskeruddannelser        39 år  2019   
18342  female  H80 Ph.d. og forskeruddannelser       40 år-  2019   
18343  female  H80 Ph.d. og forskeruddannelser          NaN  2019   
18344  female  H80 Ph.d. og forskeruddannelser          NaN  2019   

       uddannelsesaktivitet  
0                   93865.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                       0.0  
.

In [8]:
#Creating dropdown options
ALL = 'ALL'
# defning each option in the data set as unique and adding an option for all.
def unique_sorted_values_plus_all(array):
    liste = array.values.tolist()
    unique = np.unique(liste)
    unique.sort()
    unique = unique.tolist()
    unique.insert(0,ALL)
    return unique

In [9]:
# Generating dropdown menues
dropdown_year = widgets.Dropdown(options=dataproject.unique_sorted_values_plus_all(df_long.year))
dropdown_age = widgets.Dropdown(options=dataproject.unique_sorted_values_plus_all(df_long.age))
dropdown_sex = widgets.Dropdown(options=dataproject.unique_sorted_values_plus_all(df_long.sex))
dropdown_education = widgets.Dropdown(options=dataproject.unique_sorted_values_plus_all(df_long.education))

#Defining outputs
output = widgets.Output()
plot_output = widgets.Output()

In [10]:
# Defining the function to later observe.
# these are defined in the .ipynb becuase we do not know how to pass the variables through .observe in the next cell.

# takes all the dropdowns and dataframe for the function "filters" except change instead of year so the .observe on year will look for changes in the year dropdown widget.
def dropdown_year_eventhandler(change, dropdown_age = dropdown_age, dropdown_sex = dropdown_sex, dropdown_education = dropdown_education, df_long = df_long, output = output, plot_output = plot_output):
    dataproject.filters(change.new, dropdown_age.value, dropdown_sex.value, dropdown_education.value, df_long, output, plot_output)

#same as year, but with age and so on.
def dropdown_age_eventhandler(change, dropdown_year = dropdown_year, dropdown_sex = dropdown_sex, dropdown_education = dropdown_education, df_long = df_long, output = output, plot_output = plot_output):
    dataproject.filters(dropdown_year.value, change.new, dropdown_sex.value, dropdown_education.value, df_long, output, plot_output)

def dropdown_sex_eventhandler(change, dropdown_year = dropdown_year, dropdown_age = dropdown_age, dropdown_education = dropdown_education, df_long = df_long, output = output, plot_output = plot_output):
    dataproject.filters(dropdown_year.value, dropdown_age.value, change.new, dropdown_education.value, df_long, output, plot_output)

def dropdown_education_eventhandler(change, dropdown_year = dropdown_year, dropdown_age = dropdown_age, dropdown_sex = dropdown_sex, df_long = df_long, output = output, plot_output = plot_output):
    dataproject.filters(dropdown_year.value, dropdown_age.value, dropdown_sex.value, change.new, df_long, output, plot_output)


In [11]:
#Observe changes in the dropdown.
dropdown_year.observe(
dropdown_year_eventhandler, names='value')
dropdown_age.observe(
dropdown_age_eventhandler, names='value')
dropdown_sex.observe(
dropdown_sex_eventhandler, names='value')
dropdown_education.observe(
dropdown_education_eventhandler, names='value')

In [12]:
#Display dropdown menus
display(dropdown_year)
display(dropdown_age)
display(dropdown_sex)
display(dropdown_education)

Dropdown(options=('ALL', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015…

Dropdown(options=('ALL', '-5 år', '10 år', '11 år', '12 år', '13 år', '14 år', '15 år', '16 år', '17 år', '18 …

Dropdown(options=('ALL', 'female', 'male', 'total'), value='ALL')

Dropdown(options=('ALL', 'H10 Grundskole', 'H15 Forberedende uddannelser', 'H20 Gymnasiale uddannelser', 'H29 …

In [13]:
#Display Table
display(output)
#Display graph
#The graph does not work if there are 1 or less total data points, some options have NaN and will break the graph.
#display(plot_output) # ommited because it looks ugly.

Output()

In [ ]:
data = pd.read_excel (r'Data.xlsx') # import the data
# År
years = [2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

# Total antal

antal_tot_1 = np.array(data.iloc[2,5:20])

#print(antal_tot_1)

# Alder

alder_1 = np.array(data.iloc[3:39,4])

#print(alder_1)


# Antal

i = np.array(range(15))
#print(i)
antal_test =  np.array(data.iloc[3:39,5+i])
antal = antal_test.transpose()
#print(antal)



In [ ]:
# mænd i 411
z = np.array([0, 37, 37*2, 37*3 ,37*4, 37*5, 37*6, 37*7, 37*8, 37*9, 37*10])

np.ones(11)

alder_m = np.array(data.iloc[410+z[0]:446+z[0],4])

#print(alder_m)

antal_tot_m = np.array(data.iloc[409+z,5:20])

#print(antal_tot_m)

uddannelse_m = pd.Series(['Grundskole_m','Forberedende uddannelser_m','Gymnasiale uddannelser_m','Erhvervsfaglige grundforløb_m','Erhvervsfaglige uddannelser_m','Adgangsgivende uddannelsesforløb_m','KVU_m','MVU_m','BACH_m','LVU_m','Ph.d. og forskeruddannelser_m'])
uddannelse_k = pd.Series(['Grundskole_k','Forberedende uddannelser_k','Gymnasiale uddannelser_k','Erhvervsfaglige grundforløb_k','Erhvervsfaglige uddannelser_k','Adgangsgivende uddannelsesforløb_k','KVU_k','MVU_k','BACH_k','LVU_k','Ph.d. og forskeruddannelser_k'])


In [ ]:
df_m = pd.DataFrame(
    antal_tot_m,
    columns = years,
    index = uddannelse_m
)
#print(df_m)
df_m.drop('Forberedende uddannelser_m', inplace=True)
df_m.drop('Erhvervsfaglige grundforløb_m', inplace=True)

df_m


In [ ]:

df_m.T.plot(kind = 'bar', stacked = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

plt.show()




# Kvinder 
alder_k = np.array(data.iloc[817+z[0]:853+z[0],4])

#print(alder_m)

antal_tot_k = np.array(data.iloc[816+z,5:20])

#print(antal_tot_m)


In [ ]:
df_k = pd.DataFrame(
    antal_tot_k,
    columns = years,
    index = uddannelse_k
)
#print(df_k)

# Drop
df_k.drop('Erhvervsfaglige grundforløb_k', inplace=True)
df_k.drop('Forberedende uddannelser_k',inplace=True)

df_k

In [ ]:
df_k.T.plot(kind = 'bar', stacked = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

plt.show()

In [ ]:
# Appending
df_mk = df_m.append(df_k)

df_mk

df_m.T.plot(kind = 'bar', stacked = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

df_k.T.plot(kind = 'bar', stacked = True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

plt.show()



In [ ]:
i = np.array(range(15))
#print(i)
antal_test =  np.array(data.iloc[3:39,5+i])
antal = antal_test.transpose()
#print(antal)


#c = np.array(range(36))

x_1 = np.array(range(36))
x = sorted(x_1,reverse=True)



def v(x ,c = 36):
    return np.array(range(0, c-x))
def w(x):
    return sorted(v(x), reverse=True)
#print(w(0))

#def v:
#    if v>0:
#        return ()
#    if v < 0:
#        return
    


def antal_a_2005(x, i = 0):
    return np.array(data.iloc[3+v(x),5+i])


#def antal_a_2005(v, c = c, i = 0):
#    return np.array(data.iloc[3+c-v,5+i])
